In [3]:
import os
import urllib.request
import pandas as pd #데이터프레임을 만들기 위해 임포트
import json
import ssl

client_id = "VVrSb7DXLYUr2IRlQNSU"
client_secret = "pJSQFrP13Z"
encText = urllib.parse.quote("미국")#검색할 키워드를 선택


df = pd.DataFrame()#빈 데이터프레임 생성

for page in range(1, 11):#총 1000개의 데이터를 얻기 위한 범위 설정
    url = f"https://openapi.naver.com/v1/search/book?query={encText}&display=100&start={100*(page-1)+1}"#미국을 네이버 도서에 검색하고, 100개식 보여주며, 이미지가 0, 101, 201, 301, 401, 501, 601, 701, 801, 901부터 실행되도록 설정
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    context = ssl._create_unverified_context()#ssl 인증 무시 구문을 추가함
    response = urllib.request.urlopen(request, context=context)#ssl 리퀘스트를 참조
    rescode = response.getcode()

    if(rescode==200): #url 통신이 정상적이면
        response_body = response.read()
        result = response_body.decode('utf-8')#utf-8 인코딩 방식으로 디코딩하여, 바이트 형식을 문자열로 변환한 후 result 변수에 저장
        
        book_list = json.loads(result)#json.loads() 함수를 통해 딕셔너리나 리스트 형태로 변환
        
        df_page = pd.DataFrame(book_list['items'])#페이지에서 수집한 데이터를 데이터프레임에 추가
        df = pd.concat([df, df_page], axis=0)#이전 페이지까지 수집한 데이터프레임 df와 현재 페이지에서 수집한 데이터프레임 df_page를 결합하여 새로운 데이터프레임을 만듬
        
        print(f"네이버 도서 데이터 {page}00개 수집 완료")#제대로 수집했을 시 프린트
            
    else:
        print(f"네이버 도서 데이터 {page}00개 수집 실패")#수집에 실패했을 시 프린트


df.to_csv("naver_book.csv", index=True)#데이터프레임을 csv 파일에 저장

read_data = pd.read_csv("naver_book.csv", sep=",")#데이터프레임을 읽어서 변수에 저장
print (read_data)#데이터 프레임 프린트

print("1000개 네이버 도서 데이터 수집 완료")#전부 다 끝나면 프린트


네이버 도서 데이터 100개 수집 완료
네이버 도서 데이터 200개 수집 완료
네이버 도서 데이터 300개 수집 완료
네이버 도서 데이터 400개 수집 완료
네이버 도서 데이터 500개 수집 완료
네이버 도서 데이터 600개 수집 완료
네이버 도서 데이터 700개 수집 완료
네이버 도서 데이터 800개 수집 완료
네이버 도서 데이터 900개 수집 완료
네이버 도서 데이터 1000개 수집 완료
     Unnamed: 0                                     title  \
0             0  프렌즈 미국 동부(2022~2023) (뉴욕·워싱턴 DC·보스턴·시카고)   
1             1       뉴욕 셀프트래블(2023-2024) (2023-2024 최신판)   
2             2                               다시, 서쪽으로 가다   
3             3   프렌즈 캐나다(2022~2023) (벤쿠버·토론토·몬트리올·퀘벡·로키)   
4             4  미국 서부 셀프트래블(2022-2023) (믿고 보는 해외여행 가이드북)   
..          ...                                       ...   
995          95                 미국의 대학평가 (고등교육의 질 보장을 위한)   
996          96                                미국, 이라는 문제   
997          97                                   미국영어음성학   
998          98                                  미국 장로교회사   
999          99                      미국의 예비선거 (비교정치학적 접근)   

                                              

.csv로 저장하면 가독성이 너무 떨어지고, .csv를 vscode가 아닌 엑셀로 열었을 때 글자가 깨져 아래는 .xlsx로 만드는 코드입니다.

In [4]:
import os
import urllib.request
import json
import ssl
import xlsxwriter

client_id = "VVrSb7DXLYUr2IRlQNSU"
client_secret = "pJSQFrP13Z"
encText = urllib.parse.quote("미국")

book_workbook = xlsxwriter.Workbook('naver_book.xlsx')#naver_book이라는 xlsx 엑셀 파일을 생성
book_worksheet = book_workbook.add_worksheet()#새로운 시트를 추가

headers = ['title', 'link', 'image', 'author', 'discount', 'publisher', 'pubdate', 'isbn', 'description']# 리스트 형태의 headers 변수 선언
for col, header in enumerate(headers):#enumerate함수를 사용해 리스트를 반복, col에는 인덱스 번호, header에는 해당 원소의 값이 저장
    book_worksheet.write(0, col, header)#새로 생성한 엑셀 파일의 첫 번째 행(인덱스 0)에, headers 리스트의 원소를 엑셀 파일의 각 열에 쓰기 위해 write 함수를 이용, 이때 엑셀 파일의 첫 번째 행에 쓰여질 내용은 headers 리스트의 각 원소

for page in range(1, 11):
    url = f"https://openapi.naver.com/v1/search/book?query={encText}&display=100&start={100*(page-1)+1}"
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    context = ssl._create_unverified_context()
    response = urllib.request.urlopen(request, context=context)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        result = json.loads(response_body)
        book_list = result['items']

        for i, book in enumerate(book_list, 1+100*(page-1)):#enumerate함수를 사용하여 book_list가 루프를 동면서 해당 도서 정보의 인덱스(i), 도서 정보(book)을 가져옴
            book_worksheet.write(i, 0, book['title'])
            book_worksheet.write(i, 1, book['link'])
            book_worksheet.write(i, 2, book['image'])
            book_worksheet.write(i, 3, book['author'])
            book_worksheet.write(i, 4, book['discount'])
            book_worksheet.write(i, 5, book['publisher'])
            book_worksheet.write(i, 6, book['pubdate'])
            book_worksheet.write(i, 7, book['isbn'])
            book_worksheet.write(i, 8, book['description'])
            #write 함수를 이용하여 해당 정보를 book_worksheet 작성, 첫번째 인자에 행 번호(i)를, 두번째 인자에 열 번호 저장, 세번째 인자로에 book 리스트에 (title, link, image 등)의 값을 저장
        print(f"네이버 도서 데이터 {page}00개 수집 완료")
            
    else:
        print(f"네이버 도서 데이터 {page}00개 수집 실패")
 
book_workbook.close()#close함수를 이용해 엑셀 파일을 저장
print("1000개 네이버 도서 데이터 수집 완료")


네이버 도서 데이터 100개 수집 완료
네이버 도서 데이터 200개 수집 완료
네이버 도서 데이터 300개 수집 완료
네이버 도서 데이터 400개 수집 완료
네이버 도서 데이터 500개 수집 완료
네이버 도서 데이터 600개 수집 완료
네이버 도서 데이터 700개 수집 완료
네이버 도서 데이터 800개 수집 완료
네이버 도서 데이터 900개 수집 완료
네이버 도서 데이터 1000개 수집 완료
1000개 네이버 도서 데이터 수집 완료
